lets see if llm_generate gives us a response to our prompts with only text but using `guided_choice`

In [ ]:
# Here we flatten the dictionary and build a new column "input" as a formatted multi-line string for the content.
df_mini = df_mini.with_column("input", format("""[{"type": "text", "text": "{}"}, {"type": "image_url","image_url": {"url": "data:image/png;base64,{}")}}]""",
    col("user"),
    col("image_base64"),
    )
).collect()
df_mini.show()

\Lets try passing the input column that we constructed.

In [ ]:
from daft import col
from daft.functions import llm_generate, format
content = format("""[{"type": "text", "text": "{}"}, {"type": "image_url","image_url": {"url": "data:image/png;base64,{}")}}]""",
    col("user"),
    col("image_base64"),
    )
df_mini_result = df_mini.with_column("result", llm_generate(
        " ",
        model=model_id,
        provider="openai",
        extra_body={"guided_choice": ["A", "B", "C", "D"]},
        base_url=base_url,
        api_key=api_key,
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": col("user")},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/png;base64,{col("image_base64")}"},
                    },
                ],
            }
        ],
    )
).collect()

# Nope... we get a cryptic error

Hold on... lets run the full gamut of structured outputs just to see what works...

yesterday I was running into errors with pydantic json_schema try the simple pydantic schema with this dataset just for kicks.

At this point I'm pretty unsure of how to move forward with passing an image into llm_generate. Especially for this use case where we need to `format` both the prompt and image columns together into a dictionary where input_column expects to be a string.  